In [ ]:
%pip install openai
%pip install openai-agents 'openai-agents[voice]'
%pip install numpy
%pip install sounddevice
%pip install os
%pip install dotenv

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

from agents import Agent, function_tool,  FileSearchTool
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

VECTOR_STORE_ID = os.getenv("VECTOR_STORE_ID")


## Agent Setup

In [10]:
# File Agent

knowledge_agent = Agent(
    name="AnytimeFitnessKnowledgeAgent",
    instructions=(
        "You are a friendly and knowledgeable representative for Anytime Fitness. "
        "Your sole purpose is to answer user questions about Anytime Fitness by using the FileSearchTool. "
        "Provide clear, helpful, and concise answers based strictly on the information found in the documents. "
        "Do not make up information."
    ),
    tools= [FileSearchTool(
        vector_store_ids=[VECTOR_STORE_ID]
        )],
)

print(knowledge_agent)

Agent(name='AnytimeFitnessKnowledgeAgent', instructions='You are a friendly and knowledgeable representative for Anytime Fitness. Your sole purpose is to answer user questions about Anytime Fitness by using the FileSearchTool. Provide clear, helpful, and concise answers based strictly on the information found in the documents. Do not make up information.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FileSearchTool(vector_store_ids=['vs_684abbbf22808191a931c6d613d15f5a'], max_num_results=None, include_search_results=False, ranking_options=None, filters=None)], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_beha

In [11]:
# Triage Agent (not necessary for now but for further expansion)

triage_agent = Agent(
    name="Assistant",
    instructions=prompt_with_handoff_instructions(
        """You are the virtual assistant for Anytime Fitness. 
        Welcome the user warmly and determine their needs.
        Based on their query, route to the correct specialist:

        - AnytimeFitnessKnowledgeAgent for any questions about Anytime Fitness memberships, equipment, policies, or general information."""
    ),
    # This list tells the Triage Agent which specialists it can hand off to.
    handoffs=[knowledge_agent],
)

print(triage_agent)

Agent(name='Assistant', instructions='# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n\n\nYou are the virtual assistant for Anytime Fitness. \n        Welcome the user warmly and determine their needs.\n        Based on their query, route to the correct specialist:\n\n        - AnytimeFitnessKnowledgeAgent for any questions about Anytime Fitness memberships, equipment, policies, or general information.', handoff_description=None, handoffs=[Agent(name='AnytimeFitnessKnowledgeAgent

## Voice Pipeline

In [ ]:
import numpy as np
import sounddevice as sd
from agents.voice import AudioInput, SingleAgentVoiceWorkflow, VoicePipeline
from agents import trace
import asyncio 


workflow = SingleAgentVoiceWorkflow(agent=triage_agent)
pipeline = VoicePipeline(workflow=workflow)
samplerate = int(sd.query_devices(kind='input')['default_samplerate'])

async def main():
    print("🎤 Agent is ready. Type 'exit' and press Enter at the prompt to quit.")
    print("-" * 70)

    # main loop
    while True:
        cmd = input("Press Enter to speak...")
        if cmd.lower() == 'exit':
            print("👋 Goodbye!")
            break

        print("🎤 Listening... Press Enter again when you are finished speaking.")
        
        recorded_chunks = []
        def callback(indata, frames, time, status):
            if status:
                print(f"🔴 Recording error: {status}", flush=True)
            recorded_chunks.append(indata.copy())
            
        with sd.InputStream(callback=callback, samplerate=samplerate, channels=1, dtype='int16'):
            input()
            
        print("Processing...")
        recording = np.concatenate(recorded_chunks, axis=0)
        

        audio_input = AudioInput(buffer=recording, frame_rate=samplerate)
        
        with trace("Anytime Fitness Voice Agent"):
            result = await pipeline.run(audio_input)
        
        response_chunks = []
        async for event in result.stream():
            if event.type == "voice_stream_event_audio":
                response_chunks.append(event.data)
                
        if response_chunks:
            print("🤖 Assistant is responding...")
            response_audio = np.concatenate(response_chunks, axis=0)
            
            sd.play(response_audio, samplerate=samplerate)
            sd.wait()
            print("Done")

if __name__ == "__main__":
    try:
        await main()
    except KeyboardInterrupt:
        print("\nExiting program.")

🎤 Agent is ready. Type 'exit' and press Enter at the prompt to quit.
----------------------------------------------------------------------
🎤 Listening... Press Enter again when you are finished speaking.
Processing...
🎤 Listening... Press Enter again when you are finished speaking.
Processing...
